<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.listdir('./data')

['test.csv',
 'bus_bts.csv',
 '행정_법정동 중심좌표.xlsx',
 'submission_sample.csv',
 'train.csv']

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/submission_sample.csv')

In [4]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
test.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff
0,415423,2019-10-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,4.0,4.0,7.0,2.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,415424,2019-10-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,6.0,6.0,1.0,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0
2,415425,2019-10-01,4270000,시외,432,정존마을,33.48181,126.47352,2.0,4.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,415426,2019-10-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,1.0,11.0,18.0,8.0,26.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
4,415427,2019-10-01,4270000,시내,1636,롯데호텔,33.24872,126.41032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
submission.head()

,id,18~20_ride
0,415423,0
1,415424,0
2,415425,0
3,415426,0
4,415427,0


In [7]:
set(train.columns) - set(test.columns)

{'18~20_ride'}

In [8]:
train['date'].agg(['min', 'max'])  # 날짜의 최솟값, 최댓값 확인

min    2019-09-01
max    2019-09-30
Name: date, dtype: object

In [9]:
test['date'].agg(['min', 'max'])

min    2019-10-01
max    2019-10-16
Name: date, dtype: object

In [10]:
train.isna().sum()  # 결측치 확인

id               0
date             0
bus_route_id     0
in_out           0
station_code     0
station_name     0
latitude         0
longitude        0
6~7_ride         0
7~8_ride         0
8~9_ride         0
9~10_ride        0
10~11_ride       0
11~12_ride       0
6~7_takeoff      0
7~8_takeoff      0
8~9_takeoff      0
9~10_takeoff     0
10~11_takeoff    0
11~12_takeoff    0
18~20_ride       0
dtype: int64

In [11]:
test.isna().sum()

id               0
date             0
bus_route_id     0
in_out           0
station_code     0
station_name     0
latitude         0
longitude        0
6~7_ride         0
7~8_ride         0
8~9_ride         0
9~10_ride        0
10~11_ride       0
11~12_ride       0
6~7_takeoff      0
7~8_takeoff      0
8~9_takeoff      0
9~10_takeoff     0
10~11_takeoff    0
11~12_takeoff    0
dtype: int64

In [12]:
y_train = train['18~20_ride']  # target 지정

In [13]:
# train, test 합치기
data = pd.concat([train.drop('18~20_ride', axis=1), test]).reset_index(drop=True)

In [14]:
print(train.shape)
print(test.shape)
print(data.shape)

(415423, 21)
(228170, 20)
(643593, 20)


In [15]:
# 컬럼별 유니크한 값의 개수
for col in data.columns:
    print(f'column: {col}')
    print(f'# of unique values: {data[col].nunique()}')
    print()

column: id
# of unique values: 643593

column: date
# of unique values: 46

column: bus_route_id
# of unique values: 631

column: in_out
# of unique values: 2

column: station_code
# of unique values: 3601

column: station_name
# of unique values: 1971

column: latitude
# of unique values: 3310

column: longitude
# of unique values: 3481

column: 6~7_ride
# of unique values: 48

column: 7~8_ride
# of unique values: 83

column: 8~9_ride
# of unique values: 84

column: 9~10_ride
# of unique values: 71

column: 10~11_ride
# of unique values: 69

column: 11~12_ride
# of unique values: 77

column: 6~7_takeoff
# of unique values: 35

column: 7~8_takeoff
# of unique values: 56

column: 8~9_takeoff
# of unique values: 53

column: 9~10_takeoff
# of unique values: 51

column: 10~11_takeoff
# of unique values: 51

column: 11~12_takeoff
# of unique values: 58



In [16]:
# 요일 변수 생성 및 원핫인코딩
data['date'] = pd.to_datetime(data['date'])
data['weekday'] = data['date'].dt.weekday
data = pd.get_dummies(data, columns=['weekday'])

In [17]:
# 시내/시외 정수 맵핑
data['in_out'].value_counts()

시내    632939
시외     10654
Name: in_out, dtype: int64

In [18]:
data['in_out'] = data['in_out'].map({'시내': 0, '시외': 1})

In [19]:
# 승하차 시간대를 2시간으로 통합
data['6~8_ride'] = data['6~7_ride'] + data['7~8_ride']
data['8~10_ride'] = data['8~9_ride'] + data['9~10_ride']
data['10~12_ride'] = data['10~11_ride'] + data['11~12_ride']
data['6~8_takeoff'] = data['6~7_takeoff'] + data['7~8_takeoff']
data['8~10_takeoff'] = data['8~9_takeoff'] + data['9~10_takeoff']
data['10~12_takeoff'] = data['10~11_takeoff'] + data['11~12_takeoff']

In [20]:
data.columns

Index(['id', 'date', 'bus_route_id', 'in_out', 'station_code', 'station_name',
       'latitude', 'longitude', '6~7_ride', '7~8_ride', '8~9_ride',
       '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff',
       '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
       'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', '6~8_ride', '8~10_ride', '10~12_ride',
       '6~8_takeoff', '8~10_takeoff', '10~12_takeoff'],
      dtype='object')

In [21]:
# 학습에 활용할 변수 선택
input_var = ['bus_route_id', 'in_out', 'station_code', 'latitude', 'longitude',
             'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
             '6~8_ride', '8~10_ride', '10~12_ride', '6~8_takeoff', '8~10_takeoff', '10~12_takeoff']

In [22]:
data = data[input_var]

In [23]:
data.head()

,bus_route_id,in_out,station_code,latitude,longitude,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,6~8_ride,8~10_ride,10~12_ride,6~8_takeoff,8~10_takeoff,10~12_takeoff
0,4270000,1,344,33.48990,126.49373,0,0,0,0,0,0,1,1.0,7.0,8.0,0.0,0.0,0.0
1,4270000,1,357,33.48944,126.48508,0,0,0,0,0,0,1,5.0,6.0,11.0,0.0,0.0,0.0
2,4270000,1,432,33.48181,126.47352,0,0,0,0,0,0,1,2.0,2.0,0.0,0.0,0.0,0.0
3,4270000,0,1579,33.50577,126.49252,0,0,0,0,0,0,1,17.0,32.0,30.0,0.0,0.0,0.0
4,4270000,0,1646,33.25579,126.41260,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
# train, test 나누기
X_train = data[:len(train)]
X_test = data[len(train):].reset_index(drop=True)

In [25]:
# RandomForest
rf = RandomForestRegressor(n_estimators=500, random_state=2021)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=2021, verbose=0, warm_start=False)

In [26]:
pred = rf.predict(X_test)
submission['18~20_ride'] = pred
submission.to_csv('./submission/baseline_rf_500.csv', index=False)